In [125]:
import folium
import pandas as pd
from folium.plugins import BeautifyIcon
import geopandas as gpd

import matplotlib.pyplot as plt
# import contextily as ctx

<h2>Collect data from AEC, save it to current directory</h2>

In [130]:
booths = pd.read_csv('polling_booths_2022.csv')
tcp = pd.read_csv('HouseTcpByCandidateByPollingPlaceDownload-27966_2.csv')

In [132]:
tcp.head()

,StateAb,DivisionID,DivisionNm,PollingPlaceID,PollingPlace,CandidateID,Surname,GivenNm,BallotPosition,Elected,HistoricElected,PartyAb,PartyNm,OrdinaryVotes,Swing
0,ACT,318,Bean,93925,Belconnen BEAN PPVC,36231,SMITH,David,3,N,Y,ALP,Australian Labor Party,565,3.36
1,ACT,318,Bean,93925,Belconnen BEAN PPVC,37198,HIATT,Jane,6,N,N,LP,Liberal,338,-3.36
2,ACT,318,Bean,11877,Bonython,36231,SMITH,David,3,N,Y,ALP,Australian Labor Party,798,0.38
3,ACT,318,Bean,11877,Bonython,37198,HIATT,Jane,6,N,N,LP,Liberal,479,-0.38
4,ACT,318,Bean,11452,Calwell,36231,SMITH,David,3,N,Y,ALP,Australian Labor Party,1065,3.48


<h2>From the AEC data, extract the Divisions of Wills and Cooper</h2>

In [133]:
wills_booths = booths.loc[booths["DivisionNm"]=="Wills"][["PollingPlaceNm","Latitude","Longitude"]].reset_index()
cooper_booths = booths.loc[booths["DivisionNm"]=="Cooper"][["PollingPlaceNm","Latitude","Longitude"]].reset_index()

wills_tcp = tcp.loc[tcp["DivisionNm"]=="Wills"][["PollingPlace","Surname","OrdinaryVotes","Swing"]].reset_index()
cooper_tcp = tcp.loc[tcp["DivisionNm"]=="Cooper"][["PollingPlace","Surname","OrdinaryVotes","Swing"]].reset_index()

In [23]:
myIcon = folium.CustomIcon('Maps-256.png',icon_size = (30, 30),icon_anchor = (15, 30),popup_anchor = (30,30))

FileNotFoundError: [Errno 2] No such file or directory: 'Maps-256.png'

In [134]:
wills_tcp_k = wills_tcp.drop(wills_tcp[wills_tcp["Surname"]=="JEFFORD"].index)
wills_tcp_j = wills_tcp.drop(wills_tcp[wills_tcp["Surname"]=="KHALIL"].index)
wills_tcp_kj = pd.merge(wills_tcp_k,wills_tcp_j,left_on="PollingPlace",right_on="PollingPlace")

In [135]:
wills = pd.merge(wills_booths,wills_tcp_kj,left_on="PollingPlaceNm",right_on="PollingPlace")
wills = wills.drop(columns=["index","PollingPlaceNm","index_x","index_y"])
wills = wills.drop(wills[wills["OrdinaryVotes_x"]==0].index)
wills.head()

,Latitude,Longitude,PollingPlace,Surname_x,OrdinaryVotes_x,Swing_x,Surname_y,OrdinaryVotes_y,Swing_y
0,-37.771500,144.960000,Brunswick,KHALIL,891,-1.57,JEFFORD,1212,1.57
1,-37.772788,144.980243,Brunswick East (Wills),KHALIL,409,-5.82,JEFFORD,586,5.82
2,-37.759900,144.951000,Brunswick North,KHALIL,1290,-3.13,JEFFORD,1449,3.13
3,-37.765203,144.979674,Brunswick North East,KHALIL,864,-2.57,JEFFORD,1177,2.57
4,-37.756987,144.944426,Brunswick North West,KHALIL,504,-2.55,JEFFORD,543,2.55


In [136]:
cooper_tcp_l = cooper_tcp.drop(cooper_tcp[cooper_tcp["Surname"]=="KEARNEY"].index)
cooper_tcp_k = cooper_tcp.drop(cooper_tcp[cooper_tcp["Surname"]=="LIDDLE"].index)
cooper_tcp_lk = pd.merge(cooper_tcp_k,cooper_tcp_l,left_on="PollingPlace",right_on="PollingPlace")

In [137]:
cooper = pd.merge(cooper_booths,cooper_tcp_lk,left_on="PollingPlaceNm",right_on="PollingPlace")
cooper = cooper.drop(columns=["index","PollingPlaceNm","index_x","index_y"])
cooper = cooper.drop(cooper[cooper["OrdinaryVotes_x"]==0].index)
cooper.head()

,Latitude,Longitude,PollingPlace,Surname_x,OrdinaryVotes_x,Swing_x,Surname_y,OrdinaryVotes_y,Swing_y
0,-37.783697,145.031895,Alphington,KEARNEY,738,-1.34,LIDDLE,473,1.34
1,-37.747551,144.990480,Bell,KEARNEY,980,-6.49,LIDDLE,795,6.49
2,-37.707078,145.062267,Bundoora (Cooper),KEARNEY,186,-7.25,LIDDLE,122,7.25
3,-37.714310,145.050160,Bundoora South,KEARNEY,509,-7.87,LIDDLE,263,7.87
4,-37.730104,144.968771,Coburg Lake (Cooper),KEARNEY,150,-13.52,LIDDLE,162,13.52


<h2>Now start the mapping, using the data from the previous dataframes</h2>

In [138]:
vic = gpd.read_file("E_VIC21_region.shp")
wills_shp = vic.loc[vic["Elect_div"]=="Wills"]
wills_json= wills_shp.to_json()
wills_j = folium.GeoJson(data=wills_json,style_function=lambda x: {'fillColor': 'orange'})

In [147]:
wills_shp

,E_div_numb,Elect_div,Numccds,Actual,Projected,Total_Popu,Australian,Area_SqKm,Sortname,geometry
38,39,Wills,348,108904,116877,0,0,46.44,Wills,"POLYGON Z ((144.97668 -37.77322 0.00000, 144.9..."


In [139]:
# min_lat=-37.80, max_lat=-37.75,min_long=144.85,max_long=145.05

bounds = wills_shp.total_bounds  # array([144.898012, -37.779319, 144.986865, -37.69094 ])
extent = 0.1
centre = [(bounds[1]+bounds[3])/2,(bounds[0]+bounds[2])/2]

m = folium.Map(location=centre, min_lat=bounds[1]-extent, max_lat=bounds[3]+extent,
               min_long=bounds[0]-extent, max_long=bounds[2]+extent,
               width=750,height=1000,zoom_start=13)
wills_j.add_to(m)

In [140]:
for index,row in wills.iterrows():
    alp_votes = row["OrdinaryVotes_x"]
    grn_votes = row["OrdinaryVotes_y"]
    ratio = alp_votes/grn_votes
    ttip1 = (
        "<b>{s1}:</b><br>"
        "ALP:\u00A0{v1},\u00A0Greens:\u00A0{v2}<br>"
        "Swing\u00A0to\u00A0Labor:\u00A0{s2}"   
    ).format(s1 = row["PollingPlace"],s2 = str(row["Swing_x"])+"%",v1 = str(alp_votes),v2 = str(grn_votes))
    ttip2 = (
        "<b>{s1}</b><br>"
        "Greens:\u00A0{v2},\u00A0ALP:\u00A0{v1}<br>"
        "Swing\u00A0to\u00A0Greens:\u00A0{s2}"   
    ).format(s1 = row["PollingPlace"],s2 = str(row["Swing_y"])+"%",v1 = str(alp_votes),v2 = str(grn_votes))
    if ratio > 1:
        folium.CircleMarker(location=[row["Latitude"],row["Longitude"]], popup=ttip1,
                       radius=5*ratio,color='red',fill=True,fill_color='red',fill_opacity=0.5).add_to(m)
    else:
        folium.CircleMarker(location=[row["Latitude"],row["Longitude"]], popup=ttip2,
                       radius=5/ratio,color='green',fill=True,fill_color='green',fill_opacity=0.5).add_to(m)   

In [142]:
m

In [228]:
m.save("wills.html")

In [141]:
wills_shp.total_bounds

array([144.898012, -37.779319, 144.986865, -37.69094 ])

In [146]:
wills_shp.

/tmp/amca/ipykernel_3139/3028991291.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  wills_shp.centroid


38    POINT (144.94862 -37.73013)
dtype: geometry

In [148]:
(bounds[0]+bounds[2])/2

144.94243849999998

In [149]:
(bounds[1]+bounds[3])/2

-37.7351295

In [150]:
centre

[-37.73, 144.95]

In [14]:
cooper_shp = vic.loc[vic["Elect_div"]=="Cooper"]
cooper_json= cooper_shp.to_json()
cooper_j = folium.GeoJson(data=cooper_json,style_function=lambda x: {'fillColor': 'orange'})

In [15]:
bounds_c = cooper_shp.total_bounds  # array([144.898012, -37.779319, 144.986865, -37.69094 ])
extent = 0.1
centre_c = [(bounds_c[1]+bounds_c[3])/2,(bounds_c[0]+bounds_c[2])/2]

m2 = folium.Map(location=centre_c, min_lat=bounds_c[1]-extent, max_lat=bounds_c[3]+extent,
               min_long=bounds_c[0]-extent, max_long=bounds_c[2]+extent,
               width=750,height=1000,zoom_start=13)
cooper_j.add_to(m2)

In [16]:
for index,row in cooper.iterrows():
    alp_votes = row["OrdinaryVotes_x"]
    grn_votes = row["OrdinaryVotes_y"]
    ratio = alp_votes/grn_votes
    ttip1 = (
        "<b>{s1}:</b><br>"
        "ALP:\u00A0{v1},\u00A0Greens:\u00A0{v2}<br>"
        "Swing\u00A0to\u00A0Labor:\u00A0{s2}"   
    ).format(s1 = row["PollingPlace"],s2 = str(row["Swing_x"])+"%",v1 = str(alp_votes),v2 = str(grn_votes))
    ttip2 = (
        "<b>{s1}</b><br>"
        "Greens:\u00A0{v2},\u00A0ALP:\u00A0{v1}<br>"
        "Swing\u00A0to\u00A0Greens:\u00A0{s2}"   
    ).format(s1 = row["PollingPlace"],s2 = str(row["Swing_y"])+"%",v1 = str(alp_votes),v2 = str(grn_votes))
    if ratio > 1:
        folium.CircleMarker(location=[row["Latitude"],row["Longitude"]], popup=ttip1,
                       radius=5*ratio,color='red',fill=True,fill_color='red',fill_opacity=0.5).add_to(m2)
    else:
        folium.CircleMarker(location=[row["Latitude"],row["Longitude"]], popup=ttip2,
                       radius=5/ratio,color='green',fill=True,fill_color='green',fill_opacity=0.5).add_to(m2)   

In [17]:
m2

In [231]:
m2.save('cooper.html')

<h2>Now for Voronoi triangulation with <tt>geovoronoi</tt></h2>

In [19]:
from geovoronoi.plotting import subplot_for_map, plot_voronoi_polys_with_points_in_area
from geovoronoi import voronoi_regions_from_coords, points_to_coords

In [53]:
wills_coords = wills_interior_booths[["Latitude","Longitude"]].to_numpy()
wills_coords

array([[-37.7715    , 144.96      ],
       [-37.7599    , 144.951     ],
       [-37.76520324, 144.97967392],
       [-37.75698684, 144.94442595],
       [-37.77430081, 144.95263288],
       [-37.77063355, 144.94351443],
       [-37.76208644, 144.943297  ],
       [-37.74101867, 144.96600855],
       [-37.74675058, 144.97827416],
       [-37.7217    , 144.959     ],
       [-37.73535428, 144.95196845],
       [-37.74321154, 144.96439947],
       [-37.74705375, 144.94774392],
       [-37.70251968, 144.97177875],
       [-37.7159    , 144.974     ],
       [-37.69572364, 144.96670414],
       [-37.70488104, 144.92675606],
       [-37.7012    , 144.943     ],
       [-37.69949402, 144.91959546],
       [-37.7056684 , 144.90810875],
       [-37.71624073, 144.93576004],
       [-37.75561074, 144.96688123],
       [-37.7582    , 144.968     ],
       [-37.7156    , 144.914     ],
       [-37.7198    , 144.928     ],
       [-37.72844186, 144.93694654],
       [-37.74090372, 144.94307538],
 

In [88]:
wills_shp.to_crs(epsg=3395)

,E_div_numb,Elect_div,Numccds,Actual,Projected,Total_Popu,Australian,Area_SqKm,Sortname,geometry
38,39,Wills,348,108904,116877,0,0,46.44,Wills,"POLYGON Z ((16138731.190 -4521261.465 0.000, 1..."


In [69]:
wills_booths_gpd = gpd.GeoDataFrame(
    wills_booths, geometry=gpd.points_from_xy(wills_booths.Longitude, wills_booths.Latitude))
wills_booths_gpd.

,index,PollingPlaceNm,Latitude,Longitude,geometry
0,7580,Brunswick,-37.771500,144.960000,POINT (144.96000 -37.77150)
1,7581,Brunswick East (Wills),-37.772788,144.980243,POINT (144.98024 -37.77279)
2,7582,Brunswick North,-37.759900,144.951000,POINT (144.95100 -37.75990)
3,7583,Brunswick North East,-37.765203,144.979674,POINT (144.97967 -37.76520)
4,7584,Brunswick North West,-37.756987,144.944426,POINT (144.94443 -37.75699)
5,7585,Brunswick South,-37.774301,144.952633,POINT (144.95263 -37.77430)
6,7586,Brunswick South East (Wills),-37.778701,144.972850,POINT (144.97285 -37.77870)
7,7587,Brunswick South West,-37.770634,144.943514,POINT (144.94351 -37.77063)
8,7588,Brunswick West,-37.762086,144.943297,POINT (144.94330 -37.76209)
9,7589,Coburg,-37.741019,144.966009,POINT (144.96601 -37.74102)


In [120]:
wills_shp.geometry.to_numpy

<bound method IndexOpsMixin.to_numpy of 38    POLYGON Z ((144.97668 -37.77322 0.00000, 144.9...
Name: geometry, dtype: geometry>

In [93]:
wills_shp_crs = wills_shp.to_crs(epsg=4326)
wills_shp_crs

,E_div_numb,Elect_div,Numccds,Actual,Projected,Total_Popu,Australian,Area_SqKm,Sortname,geometry
38,39,Wills,348,108904,116877,0,0,46.44,Wills,"POLYGON Z ((144.97669 -37.77321 0.00000, 144.9..."


In [116]:
nw = wills_shp_crs.to_numpy()
nw[0,4]

116877

In [78]:
wills_booths_crs = wills_booths_gpd.set_crs(epsg=3395)

In [81]:
wills_interior_booths = wills_booths_crs[wills_booths_crs.geometry.within(wills_shp_crs.geometry)]

/usr/lib/python3.10/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


In [83]:
wills_polys, wills_pts = voronoi_regions_from_coords(wills_coords, wills_shp_crs, per_geom=False)

ValueError: `geo_shape` must be a Polygon or MultiPolygon

In [28]:
wills_booths

,index,PollingPlaceNm,Latitude,Longitude
0,7580,Brunswick,-37.771500,144.960000
1,7581,Brunswick East (Wills),-37.772788,144.980243
2,7582,Brunswick North,-37.759900,144.951000
3,7583,Brunswick North East,-37.765203,144.979674
4,7584,Brunswick North West,-37.756987,144.944426
5,7585,Brunswick South,-37.774301,144.952633
6,7586,Brunswick South East (Wills),-37.778701,144.972850
7,7587,Brunswick South West,-37.770634,144.943514
8,7588,Brunswick West,-37.762086,144.943297
9,7589,Coburg,-37.741019,144.966009


In [44]:
wills_interior_booths = wills_booths.copy()

In [45]:
wills_interior_booths = wills_interior_booths.drop([1,6,11,16,17,18,27,28,31,32,40,41])

In [46]:
wills_interior_booths

,index,PollingPlaceNm,Latitude,Longitude
0,7580,Brunswick,-37.771500,144.960000
2,7582,Brunswick North,-37.759900,144.951000
3,7583,Brunswick North East,-37.765203,144.979674
4,7584,Brunswick North West,-37.756987,144.944426
5,7585,Brunswick South,-37.774301,144.952633
7,7587,Brunswick South West,-37.770634,144.943514
8,7588,Brunswick West,-37.762086,144.943297
9,7589,Coburg,-37.741019,144.966009
10,7590,Coburg East,-37.746751,144.978274
12,7592,Coburg North,-37.721700,144.959000


In [49]:
# min_lat=-37.80, max_lat=-37.75,min_long=144.85,max_long=145.05

bounds = wills_shp.total_bounds  # array([144.898012, -37.779319, 144.986865, -37.69094 ])
extent = 0.1
centre = [(bounds[1]+bounds[3])/2,(bounds[0]+bounds[2])/2]

m3 = folium.Map(location=centre, min_lat=bounds[1]-extent, max_lat=bounds[3]+extent,
               min_long=bounds[0]-extent, max_long=bounds[2]+extent,
               width=750,height=1000,zoom_start=13)
wills_j.add_to(m3)

In [52]:
for index,row in wills_interior_booths.iterrows():
    folium.CircleMarker(location=[row["Latitude"],row["Longitude"]], 
                       radius=10,color='red',fill=True,fill_color='red',fill_opacity=0.5).add_to(m3)
m3

In [54]:
voronoi_regions_from_coords?

In [56]:
wills_shp.

In [67]:
wills_booths

,index,PollingPlaceNm,Latitude,Longitude
0,7580,Brunswick,-37.771500,144.960000
1,7581,Brunswick East (Wills),-37.772788,144.980243
2,7582,Brunswick North,-37.759900,144.951000
3,7583,Brunswick North East,-37.765203,144.979674
4,7584,Brunswick North West,-37.756987,144.944426
5,7585,Brunswick South,-37.774301,144.952633
6,7586,Brunswick South East (Wills),-37.778701,144.972850
7,7587,Brunswick South West,-37.770634,144.943514
8,7588,Brunswick West,-37.762086,144.943297
9,7589,Coburg,-37.741019,144.966009


In [58]:
gpd.points_from_xy?

In [105]:
wills_shp.geometry.transform?